## Import libraries

In [2]:
from matplotlib import pyplot as plt
import json
import requests
import pprint as pprint
from datetime import datetime
from datetime import date
import ipywidgets as widgets
import openpyxl
from openpyxl import Workbook
import pandas as pd
from openpyxl import load_workbook
import os
import xlsxwriter
import xlrd

## Create variables-file (if not exists)

In [3]:
if not os.path.exists('.variables'):
    key = input("Enter your API-key: ")
    token = input("Enter your API-token: ")
    allboards = json.loads(json.dumps(requests.get("https://api.trello.com/1/members/me/boards?key="+key+"&token="+token).json()))
    print('These are your boards: ')
    for i in allboards:
        if i['closed'] == False:
            print('Name: ',i['name'],', ID:',i['id'])
    boardid = input("Enter your board-id: ")
    
    print('These are the lists on the chosen board.')
    alllists = json.loads(json.dumps(requests.get("https://api.trello.com/1/boards/"+boardid+"/lists?key="+key+"&token="+token).json()))
    for i in alllists:
        if i['closed'] == False:
            print(i['name'])
    print()
    startlists = input("Enter the lists with cards that are not started (comma separated): ")
    print()
    blockedlists = input("Enter the lists with cards that are blocked(comma separated): ")
    print()
    doinglists = input("Enter the lists with the doing cards (comma separated): ")
    print()
    donelists = input("Enter the lists with cards that are done (comma separated): ")
    print()
    print()
    filenamewithoutextension = input("Enter the name for the excel file (without .xlsx)")
    filenamewithextension = filenamewithoutextension+".xlsx"
    print()
    print()
    file = open('.variables', 'w')
    file = open('.variables', 'a')
    file.write(key)
    file.write('\n')
    file.write(token)
    file.write('\n')
    file.write(boardid)
    file.write('\n')
    file.write(startlists)
    file.write('\n')
    file.write(blockedlists)
    file.write('\n')
    file.write(doinglists)
    file.write('\n')
    file.write(donelists)
    file.write('\n')
    file.write(filenamewithextension)
    file.close()
    print('Your variables are saved in the file \'.variables\'. Please do not delete this file.')

## Get variables from file

In [2]:
f = open('.variables', 'r')
variables = f.read().splitlines()

lijstenbeginnen = []
lijstenblocked = []
lijstendoing = []
lijstendone = []




api_key = variables[0]
api_token = variables[1]
bordid = variables[2]
lijstenbeginnen = variables[3].split(',')
lijstenblocked = variables[4].split(',')
lijstendoing = variables[5].split(',')
lijstendone = variables[6].split(',')
excelfile = variables[7]

lijstenforscrum = []
for i in lijstenbeginnen:
    lijstenforscrum.append(i)
    
lijstenforscrum.extend(lijstenblocked)
lijstenforscrum.extend(lijstendoing)
lijstenforscrum.extend(lijstendone)


f.close()


## Create URLs

In [3]:
keys = "key="+api_key+"&token="+api_token
trello_base_url = "https://api.trello.com/1/"
board_url = trello_base_url+"boards/"+bordid
url_cards = board_url+"/cards?customFieldItems=true&"+keys
url_lists = board_url+"/lists?"+keys
url_customfields = board_url+"/customFields?"+keys
url_labels = board_url+"/labels?"+keys
url_members = board_url+"/members?"+keys
statussen = ['Nog starten','Blocked','Doing','Done']

## Get JSON and parse to objects 

In [4]:
cards = json.loads(json.dumps(requests.get(url_cards).json()))
lists = json.loads(json.dumps(requests.get(url_lists).json()))
customfields = json.loads(json.dumps(requests.get(url_customfields).json()))
labels = json.loads(json.dumps(requests.get(url_labels).json()))
members = json.loads(json.dumps(requests.get(url_members).json()))

In [12]:
for i in cards:
    try:
        print (i['badges']['attachmentsByType']['trello']['card'],i['name'])
    except: 
        pass

0 Sjabloon: Nieuw project
0 Training op maat
0 Alle documenten digitaliseren in suite
0 Fiattering
0 contracten weco en participatie verlengen
0 Suite in Chrome laten draaien
0 omweg documenten opvragen via H-schijf
2 Instructies maken voor bestaande processen
0 Aanbesteding HH
0 Aanbesteding Ambulant Wmo
0 Poh verwerken in dashboard
0 Aanbesteding Ambulant Jeugd
0 Aanbesteding GGZ (inclusief Dyslexie)
1 Implementatie nieuwe zaaksysteem
0 Inlichtingenbureau mogelijkheden onderzoeken
0 Nieuwe huisstijl
0 Veranderopgave Inburgering
0 Nieuwe werkprocessen Participatie
0 Wijzigen woonplaatsbegiinsel
0 Richtlijnen_Nieuwsbrief_BDFS_2020
0 In productie nemen versie 7 SSD
0 Boederij de Zonnestraal
0 Boete tabblad inrichten
0 Servicenormen opstellen
0 DMO's & Werkinstructie
0 ITIL training
0 Uitwerken proces WMO loket-USD
0 Aanpassen maatwerk (fase toetsing)
0 Berichtenverkeer iwmo en ijw 2.4
0 Nieuwe werkprocessen Bijzondere Bijstand
0 Opschonen Suite
0 Omzetting HC > Welzorg
0 BISL training
0

## Create dictionary for custom fields

In [7]:
customfields_dict = {}
for i in customfields:
    customfields_dict[i['id']] = {}
    if i['type'] == 'date':
        customfields_dict[i['id']][i['name']] = {}
        customfields_dict[i['id']][i['name']]['options'] = {'id': 'date'}
       

    else:
        customfields_dict[i['id']][i['name']] = {}
        customfields_dict[i['id']][i['name']]['options'] = {}
        for j in i['options']:
            customfields_dict[i['id']][i['name']]['options'][j['id']] =  j['value']['text']

customfieldsmetdate = []
for i,j in customfields_dict.items():
        for k,l in j.items():
            try:
                if l['options']['id'] == 'date':
                    customfieldsmetdate.append(i)
            except:
                pass


## create function to calculate datetime of creation

In [8]:
def idtodate(cardid):
    hex = cardid[0:8]
    timestamp = int(hex,16)
    timedate = datetime.fromtimestamp(timestamp)
    return timedate

In [9]:
test = idtodate('5dc148039dde3e3eaf8d47d9')
print(str(test))

2019-11-05 09:59:31


## Create dictionary for cards

In [10]:
kaarten = {}

for i in cards:
        kaarten[i['id']] = {'name': i['name'],
                            'id': i['id'],
                            'idlist': i['idList'],
                            'customfields': i['customFieldItems'],
                            'customfieldvalues': {'Gebied': None, 
                                                  'Hoofdverantwoordelijke': None, 
                                                  'Prioriteit': None,
                                                  'Voltooid': None,
                                                  'Beginnen': None,
                                                  'Tijdsbelasting': None
                                                 },
                            'labels': {},
                            'members': {},
                            'sjabloon': i['isTemplate'],
                            'vervaldatum': None

                           }
        
for i,j in kaarten.items():
    date = idtodate(i)
    j['datum aanmaak'] = str(date)
    for k in lists:
        if j['idlist'] == k['id']: j['list'] = k['name'] 
if customfields_dict == {}:
    for i,j in kaarten.items():
        j['customfieldvalues'] = {}
else:

    for i,j in kaarten.items():
        for k in j['customfields']:
        
            for l,m in customfields_dict.items():

                for n,o in m.items():
                    if k['idCustomField'] not in customfieldsmetdate:
                        for p,q in o.items():
                            for r,s in q.items():
                                if r == k['idValue']:
                                    j['customfieldvalues'][n] = s
                    if k['idCustomField'] in customfieldsmetdate:
                        j['customfieldvalues']['Beginnen'] = k['value']['date'][0:10]

                        

                  
for i in cards:
    if i['due'] != None:
        kaarten[i['id']]['vervaldatum'] = i['due'][0:10]
    for j in i['labels']:
        kaarten[i['id']]['labels'][j['name']] = j['id']


for i in cards:
    for j in i['idMembers']:

        for k in members:

            if j == k['id']:
                    kaarten[i['id']]['members'][k['id']] = k['fullName']

for i,j in kaarten.items():
    if j['list'] in lijstenbeginnen:
        j['status'] = 'Niet gestart'
    elif j['list'] in lijstendoing:
        j['status'] = 'Doing'
    elif j['list'] in lijstenblocked:
        j['status'] = 'Blocked'
    elif j['list'] in lijstendone:
        j['status'] = 'Done'

for i,j in kaarten.items():
    del j['customfields']
    del j['idlist']
    del j['id']

tedeletenlijsten = []

for i in lists:
    if i['name'] not in lijstenforscrum:
        tedeletenlijsten.append(i['name'])
tedeletenkaarten = []
for i,j in kaarten.items():
    if j['sjabloon'] == True:
        tedeletenkaarten.append(i)
    elif j['list'] in tedeletenlijsten:
        tedeletenkaarten.append(i)

for i in tedeletenkaarten:
    if i in kaarten:
        del kaarten[i]


In [11]:
pprint.pprint(kaarten, indent=2)

{ '5dbb17f7e4ce455ef36d5fcf': { 'customfieldvalues': { 'Beginnen': None,
                                                       'Gebied': 'Wmo/Jeugd',
                                                       'Hoofdverantwoordelijke': 'Kelvin',
                                                       'Prioriteit': '1. '
                                                                     'Extreem',
                                                       'Tijdsbelasting': '1. '
                                                                         'Extreem',
                                                       'Voltooid': '50%'},
                                'datum aanmaak': '2019-10-31 17:20:55',
                                'labels': {},
                                'list': 'Blocked',
                                'members': { '58f5dcc3aeb458131b5d9a5f': 'Kelvin '
                                                                         'Apeldoorn',
                           

  '5dbb19f783e39288382c5080': { 'customfieldvalues': { 'Beginnen': None,
                                                       'Gebied': 'Wmo/Jeugd',
                                                       'Hoofdverantwoordelijke': 'Kelvin',
                                                       'Prioriteit': '3. '
                                                                     'Middel',
                                                       'Tijdsbelasting': '1. '
                                                                         'Extreem',
                                                       'Voltooid': None},
                                'datum aanmaak': '2019-10-31 17:29:27',
                                'labels': {},
                                'list': 'Blocked',
                                'members': { '58f5dcc3aeb458131b5d9a5f': 'Kelvin '
                                                                         'Apeldoorn'},
                            

                                'members': { '5bf80781bf57a98880ae3015': 'Roy '
                                                                         'van '
                                                                         'Rhedenburg',
                                             '5c89f602c36bdb8b786a431d': 'Gea '
                                                                         'de '
                                                                         'Boer'},
                                'name': 'Opschonen Suite',
                                'sjabloon': False,
                                'status': 'Blocked',
                                'vervaldatum': None},
  '5dbb1d3012cfc008a3b81a95': { 'customfieldvalues': { 'Beginnen': '2019-11-07',
                                                       'Gebied': 'Algemeen',
                                                       'Hoofdverantwoordelijke': 'Roy',
                                                 

                                             '5c89f602c36bdb8b786a431d': 'Gea '
                                                                         'de '
                                                                         'Boer'},
                                'name': 'Inrichten CORV',
                                'sjabloon': False,
                                'status': 'Doing',
                                'vervaldatum': '2019-12-27'},
  '5dc12e9c4ff06051fc8acae7': { 'customfieldvalues': { 'Beginnen': None,
                                                       'Gebied': 'Wmo/Jeugd',
                                                       'Hoofdverantwoordelijke': 'Kelvin',
                                                       'Prioriteit': '3. '
                                                                     'Middel',
                                                       'Tijdsbelasting': None,
                                                       'Voltoo

                                'list': 'Blocked',
                                'members': { '58f5dcc3aeb458131b5d9a5f': 'Kelvin '
                                                                         'Apeldoorn',
                                             '5bf80781bf57a98880ae3015': 'Roy '
                                                                         'van '
                                                                         'Rhedenburg',
                                             '5c89f602c36bdb8b786a431d': 'Gea '
                                                                         'de '
                                                                         'Boer',
                                             '5c8b8602529d8d57a8f75b60': 'Michel '
                                                                         'Vijver'},
                                'name': 'MDU5',
                                'sjabloon': False,
                           

                                                       'Tijdsbelasting': None,
                                                       'Voltooid': None},
                                'datum aanmaak': '2019-12-02 13:19:05',
                                'labels': {},
                                'list': 'Nieuw',
                                'members': { '58f5dcc3aeb458131b5d9a5f': 'Kelvin '
                                                                         'Apeldoorn',
                                             '5bf80781bf57a98880ae3015': 'Roy '
                                                                         'van '
                                                                         'Rhedenburg',
                                             '5c89f602c36bdb8b786a431d': 'Gea '
                                                                         'de '
                                                                         'Boer',
                        

                                                       'Voltooid': '0%'},
                                'datum aanmaak': '2019-12-19 09:08:48',
                                'labels': {},
                                'list': 'Nieuw',
                                'members': {},
                                'name': 'Suite in Chrome laten draaien',
                                'sjabloon': False,
                                'status': 'Niet gestart',
                                'vervaldatum': None},
  '5dfca39553d91438c531d534': { 'customfieldvalues': { 'Beginnen': '2020-01-20',
                                                       'Gebied': 'Participatie '
                                                                 '(Inkomen)',
                                                       'Hoofdverantwoordelijke': 'Roy',
                                                       'Prioriteit': '3. '
                                                                     'Mid

## Create dictionary for countings

In [12]:
listforlistname = []
for i in lists:
    if i['name'] not in tedeletenlijsten:
        listforlistname.append(i['name'])

listcounts = {}
for i in listforlistname:
    listcounts[i] = sum(value['list'] == i for value in kaarten.values())
listcounts

{'Nieuw': 6,
 'Wensenlijst': 1,
 'Inplannen': 11,
 'Klaar voor de start': 11,
 'Blocked': 25,
 'Doing': 14,
 'Done': 17,
 'Cognosrapporten Inplannen': 4,
 'Cognosrapporten Done': 5}

## Split cards into new dictionaries for each status

In [13]:
nietgestart = {}
blocked = {}
doing = {}
done = {}

for i,j in kaarten.items():
    if j['list'] in lijstenbeginnen:
        nietgestart[i] = j
    elif j['list'] in lijstenblocked:
        blocked[i] = j
    elif j['list'] in lijstendoing:
        doing[i] = j
    elif j['list'] in lijstendone:
        done[i] = j


## Create another dictionary for counting

In [14]:
countingcards = {}

for i,j in customfields_dict.items():
    for k,l in j.items():
        if i not in customfieldsmetdate:
            countingcards[k] = {}
            for m,n in l.items():
                for o in n.values():
                    countingcards[k][o] = 0

counters = {}
                
for i,j in countingcards.items():
    counters[i] = {}
    
    for k,l in j.items():
        for m,n in kaarten.items():
            for o,p in n.items():
                if o == 'customfieldvalues':
                    for q,r in p.items():
                        if r != None:
                            if i==q:
                                if k==r:
                                    l += 1
                                    counters[i][r] = l


## Create function to count for each status (WIP)

In [15]:
def countingperstatus(customfield, statusdict):
    customfield = {}
    for i,j in countingcards.items():
        customfield[i] = {}
        for k,l in j.items():

            for m,n in statusdict.items():
                for o,p in n.items():
                    if o == 'customfieldvalues':
                        for q,r in p.items():
                                if i == q:
                                    if k == r:
                                        l += 1
                                        customfield[i][r] = l
    return customfield

## Create function to write to the excelfile

In [16]:
def schrijfnaarexcel(countdict,excelfile,sheetname):
    if not os.path.exists(excelfile):
        workbook = xlsxwriter.Workbook(excelfile)
        worksheet = workbook.add_worksheet("Lijsten")
        for i in statussen:
            worksheet = workbook.add_worksheet(i)
        worksheet = workbook.add_worksheet("Vervallen")
#        for i in counters:
#            worksheet = workbook.add_worksheet(i)
        workbook.close()
#    xls = xlrd.open_workbook(r'werkvoorraden.xlsx', on_demand=True)
#    sheetnames = xls.sheet_names() 
    counts = []
    index = []
    columns = []
    now = datetime.now()
    counts.append(now.strftime("%d-%m-%Y %H:%M"))
    columns.append('Date')
    for i,j in countdict.items():
        columns.append(i)
        counts.append(j)
    df_columnnames = pd.DataFrame([columns])
    df_counts = pd.DataFrame([counts],columns=columns)
    book = load_workbook(excelfile)
    writer = pd.ExcelWriter(excelfile,engine='openpyxl')
    writer.book = book
    writer.sheets = {ws.title: ws for ws in book.worksheets}
    valuecount = writer.sheets[sheetname]['B1'].value
    if valuecount == 0 or valuecount == None:
        df_columnnames.to_excel(writer,sheet_name=sheetname,startrow=1, index=False,header=False)
        writer.sheets[sheetname]['A1'].value = 'Counter'
        writer.sheets[sheetname]['B1'].value = 2
    valuecount = writer.sheets[sheetname]['B1'].value
    df_counts.to_excel(writer,sheet_name=sheetname, startrow=writer.sheets[sheetname]['B1'].value, index = False,header= False)
    writer.sheets[sheetname]['B1'].value = valuecount + 1
    writer.save()

## Create objects to write to excel (WIP)

In [17]:
count_nietgestart = countingperstatus('prioriteit', nietgestart)
count_blocked = countingperstatus('prioriteit', blocked)
count_doing = countingperstatus('prioriteit', doing)
count_done = countingperstatus('prioriteit', done)


## Create function to write fields to excel (WIP)

In [18]:
def schrijffieldsnaarexcel(countdict,excelfile,sheetname):
    if not os.path.exists(excelfile):
        workbook = xlsxwriter.Workbook(excelfile)
        worksheet = workbook.add_worksheet("Lijsten")
        for i in statussen:
            worksheet = workbook.add_worksheet(i)
#        for i in counters:
#            worksheet = workbook.add_worksheet(i)
        workbook.close()
#    xls = xlrd.open_workbook(r'werkvoorraden.xlsx', on_demand=True)
#    sheetnames = xls.sheet_names() 
    counts = []
    index = []
    now = datetime.now()
    counts.append(now.strftime("%d-%m-%Y %H:%M"))
    columns.append('Date')
    for i,j in countdict.items():
        columns.append(i)
        columns2.append(' ')
        for k,l in j.items():
            columns.append(' ')
            columns2.append(k)
        counts.append(j)
    df_columnnames = pd.DataFrame([columns])
    df_counts = pd.DataFrame([counts],columns=columns)
    book = load_workbook(excelfile)
    writer = pd.ExcelWriter(excelfile,engine='openpyxl')
    writer.book = book
    writer.sheets = {ws.title: ws for ws in book.worksheets}
    valuecount = writer.sheets[sheetname]['B1'].value
    if valuecount == 0 or valuecount == None:
        df_columnnames.to_excel(writer,sheet_name=sheetname,startrow=1, index=False,header=False)
        writer.sheets[sheetname]['A1'].value = 'Counter'
        writer.sheets[sheetname]['B1'].value = 2
    valuecount = writer.sheets[sheetname]['B1'].value
    df_counts.to_excel(writer,sheet_name=sheetname, startrow=writer.sheets[sheetname]['B1'].value, index = False,header= False)
    writer.sheets[sheetname]['B1'].value = valuecount + 1
    writer.save()

## Create list for writing fields to excel (WIP)

In [19]:
listforverantwoordelijke = []
listforgebied = []
listforvoltooid = []
listfortijdsbelasting = []
listforprioriteit = []

for i,j in kaarten.items():
    if j['customfieldvalues'] != {}:
        for k,l in j['customfieldvalues'].items():
            if k == 'Gebied':
                listforgebied.append(l)
            elif k == 'Tijdsbelasting':
                listfortijdsbelasting.append(l)
            elif k == 'Prioriteit':
                listforprioriteit.append(l)
            elif k == 'Voltooid':
                listforvoltooid.append(l)
            elif k == 'Hoofdverantwoordelijke':
                listforverantwoordelijke.append(l)

listforverantwoordelijke = list(dict.fromkeys(listforverantwoordelijke))
listforgebied = list(dict.fromkeys(listforgebied))
listforvoltooid = list(dict.fromkeys(listforvoltooid))
listfortijdsbelasting = list(dict.fromkeys(listfortijdsbelasting))
listforprioriteit = list(dict.fromkeys(listforprioriteit))


## Create another dictionary for counting fields (WIP)

In [20]:
verantwoordelijkenietgestart = {}
verantwoordelijkeblocked = {}
verantwoordelijkedoing = {}
verantwoordelijkedone = {}
for i in listforverantwoordelijke:
    verantwoordelijkenietgestart[i] = sum(value['customfieldvalues']['Hoofdverantwoordelijke'] == i for value in nietgestart.values())

for i in listforverantwoordelijke:
    verantwoordelijkeblocked[i] = sum(value['customfieldvalues']['Hoofdverantwoordelijke'] == i for value in blocked.values())

for i in listforverantwoordelijke:
    verantwoordelijkedoing[i] = sum(value['customfieldvalues']['Hoofdverantwoordelijke'] == i for value in doing.values())

for i in listforverantwoordelijke:
    verantwoordelijkedone[i] = sum(value['customfieldvalues']['Hoofdverantwoordelijke'] == i for value in done.values())


## Write the counts to excel with function

In [21]:
schrijfnaarexcel(listcounts,excelfile,'Lijsten')
if customfields_dict != {}:
    schrijfnaarexcel(verantwoordelijkenietgestart,excelfile,'Nog starten')
    schrijfnaarexcel(verantwoordelijkeblocked,excelfile,'Blocked')
    schrijfnaarexcel(verantwoordelijkedoing,excelfile,'Doing')
    schrijfnaarexcel(verantwoordelijkedone,excelfile,'Done')

## Create dictionary for all cards past the due date and write to excel

In [22]:
dict_vervallen = {}

current_date = date.today()
for i,j in kaarten.items():
    if j['status'] != 'Done':
        if j['vervaldatum'] != None:
#        print(int(j['vervaldatum'][0:4]))
            vervaldatum = datetime.strptime(j['vervaldatum'], '%Y-%m-%d').date()
            delta = vervaldatum - current_date
            if delta.days <= 0:
                days = (delta.days)*-1
                if customfields_dict != {}:
                    dict_vervallen[j['name']] = {'Aantal dagen verlopen': delta.days, 
                                                    'Hoofdverantwoordelijke': j['customfieldvalues']['Hoofdverantwoordelijke'],
                                                    'Gebied': j['customfieldvalues']['Gebied']}
                else:
                    leden = []
                    for k,l in j['members'].items():
                        leden.append(l)
                        dict_vervallen[j['name']] = {'Aantal dagen verlopen': days, 
                                                    'Leden': leden}
      

book = load_workbook(excelfile)
writer = pd.ExcelWriter(excelfile,engine='openpyxl')
writer.book = book
writer.sheets = {ws.title: ws for ws in book.worksheets}
df = pd.DataFrame(data=dict_vervallen)
df.transpose().to_excel(writer, sheet_name='Vervallen')
writer.close()

TypeError: unsupported operand type(s) for -: 'datetime.date' and 'datetime.datetime'

## Calculate days on Trello-board for Done-cards

In [ ]:
df = pd.DataFrame(data=kaarten)
df.transpose()